In [ ]:
import torch
from custom_dataset import CustomImageDataset
from torch.utils.data import DataLoader
import torch.nn as nn
from timeit import default_timer as timer

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(device)

In [ ]:
batch_size=512

In [ ]:
training_dataset = CustomImageDataset(
    r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\Dataset\\Train\\annotations.csv',
    r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\Dataset\\Train',
    )

training_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

testing_dataset = CustomImageDataset(
    r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\Dataset\\Test\\annotations.csv',
    r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\Dataset\\Test'
)

testing_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def test_and_evaluate(model, device, criterion, optimizer, scheduler, epochs,  batch_size, training_loader, testing_loader, testing_dataset):
    n_total_steps = len(training_loader)
    print('Training')
    for epoch in range(epochs):
        losses = []
        start = timer()
        for i, (images, labels) in enumerate(training_loader):
            images = images.to(device)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
    
            outputs = model(images)
            loss = criterion(outputs, labels)

            losses.append(loss)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            # if (i+1) % 30 == 0:
            #     print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
        mean_loss = sum(losses) / len(losses)
        scheduler.step(mean_loss)
        end = timer()
        print(f'Time for Epoch {epoch+1}: {end-start:.4f} seconds')
    
    #Evaluate
    print('Evaluating')
    sample_limit = int(testing_dataset.__len__() / 46)
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(46)]
        n_class_samples = [0 for i in range(46)]
        for images, labels in testing_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            test_limit = min(batch_size, sample_limit)

            for i in range(test_limit):
                label = labels[i]
                pred = predicted[i]
                label = int(label)
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accurracy of the network: {acc}%')

        return acc

In [ ]:
#setup test 
result_list = []

result_fields = ['Model Name', 'Number of Epochs', 'Accuracy']

In [ ]:
main_training = False

In [ ]:
#test resnet 50
from resnet import ResNet50
learning_rate = 0.05

for num_epochs in range(10, 81, 10):
   for i in range(1):
       model = ResNet50(img_channels=1, num_classes=46).to(device)

       criterion = nn.CrossEntropyLoss()
       optimizer = torch.optim.SGD(model.parameters(), lr  = learning_rate)
       scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)

       result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)

       #store result
       temp_dict = {'Model Name': 'ResNet50','Number of Epochs': num_epochs,'Accuracy': result_accuracy}
       result_list.append(temp_dict)
       print(result_list)


In [ ]:
#write test results to csv
import csv

with open(r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\resnet50_new_learning_results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=result_fields)
    writer.writerows(result_list)

print('Saved Data!')

In [ ]:
#test resnet 101
result_list = []
from resnet import ResNet101
learning_rate = 0.05

for num_epochs in range(10, 81, 10):
    for i in range(1):
        model = ResNet101(img_channels=1, num_classes=46).to(device)
    
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr  = learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)
    
        result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
        #store result
        temp_dict = {'Model Name': 'ResNet101','Number of Epochs': num_epochs,'Accuracy': result_accuracy}
        result_list.append(temp_dict)
        print(result_list)
   

In [ ]:
#write test results to csv
import csv

with open(r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\resnet101_new_learning_results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=result_fields)
    writer.writerows(result_list)

print('Saved Data!')

In [ ]:
#test resnet 50
result_list = []
from resnet import ResNet152
learning_rate = 0.05

for num_epochs in range(10, 81, 10):
    for i in range(1):
        model = ResNet152(img_channels=1, num_classes=46).to(device)
    
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr  = learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)
    
        result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
        #store result
        temp_dict = {'Model Name': 'ResNet152','Number of Epochs': num_epochs,'Accuracy': result_accuracy}
        result_list.append(temp_dict)
        print(result_list)

In [ ]:
#write test results to csv
import csv

with open(r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\resnet152_new_learning_results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=result_fields)
    writer.writerows(result_list)

print('Saved Data!')

In [ ]:
learning_rate_list = []

In [ ]:
from resnet import ResNet50

num_epochs = 30

In [ ]:
learning_rate_test = False

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'ReduceLROnPlateau', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    l1 = lambda epoch: 0.95 ** epoch
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=l1)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'LambdaLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    
    print(learning_rate_list)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    print(count_parameters(model))
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    l1 = lambda epoch: 0.95
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=l1)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'MultiplicativeLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'StepLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'LinearLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:    
    model = ResNet50(img_channels=1, num_classes=46).to(device)

    learning_rate = 0.05

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)

    temp_dict = {
        'Learning_Model': 'ExponentialLR', 
        'Accuracy': result_accuracy
    }

    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'CosineAnnealingLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:
    model = ResNet50(img_channels=1, num_classes=46).to(device)
    
    learning_rate = 0.05
        
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, eta_min=0.0001)
    
    result_accuracy = test_and_evaluate(model, device, criterion, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    
    temp_dict = {
        'Learning_Model': 'CosineAnnealingWarmRestartsLR', 
        'Accuracy': result_accuracy
    }
    
    learning_rate_list.append(temp_dict)
    print(learning_rate_list)

In [ ]:
if learning_rate_test:
    #write test results to csv
    import csv
    
    learning_fields = ['Learning_Model', 'Accuracy']
    
    with open(r'D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\learning_results.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=learning_fields)
        writer.writerows(learning_rate_list)
    
    print('Saved Data!')